In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [73]:
postings = pd.read_csv("clean_job_postings_w_salary.csv").drop(columns = ["Unnamed: 0"])

In [74]:
skills_lang = postings[['skills', 'programming_languages', 'location']]

In [75]:
def augment_skills_location(row):
    eval_lang = eval(row['programming_languages'])#.apply(lambda x: eval(x))
    eval_skills = eval(row['skills'])#.apply(lambda x: eval(x))
    lang_str = ' '.join(eval_lang)
    skills_str = ' '.join(eval_skills)
    return lang_str + ' ' + skills_str + " " + row['location'].lower()

In [76]:
skills_lang['joined_skills'] = skills_lang.apply(augment_skills_location, axis = 1)

<ipython-input-76-b076553adac0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_lang['joined_skills'] = skills_lang.apply(augment_skills_location, axis = 1)


In [77]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(skills_lang['joined_skills'])

In [78]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [89]:
def recommend(skills, locations, similarities, jobs):
    skills_str = ' '.join(input_skills)
    locations_str = ' '.join(locations)
    desired_str = skills_str + " " + locations_str
    
    vectorized = tfidf_vectorizer.transform([desired_str])
    sim_score = cosine_similarity(vectorized, tfidf_matrix)[0]
    most_similar_jobs = sim_score.argsort()[::-1][:5]
    recommended_job = jobs.iloc[most_similar_jobs][['clean_job_title', 'location', 'clean_job_description', 
                                                    'salary estimate', 'rating']]
    return recommended_job
    

In [92]:
input_skills = ['python', 'java', 'R', 'Operations']
locations = ['Harvard']
recommendation = recommend(input_skills, locations, cosine_sim, postings)

In [93]:
recommendation

,clean_job_title,location,clean_job_description,salary estimate,rating
122,lead data scientist,Remote,we are looking for a data scientist to analyze...,$77.50 /hr (est.),2.0
673,electronic trading associate expert graduate p...,"New York, NY",description external the team that powers elec...,$140000 /yr (est.),4.0
196,data scientist operational excellence gsc,"Seattle, WA","data scientist, operational excellence gsc job...",$124950 /yr (est.),3.7
243,senior data scientist,"Santa Monica, CA",job title senior data scientist requisition id...,$132367 /yr (est.),3.5
230,senior data scientist,"Santa Monica, CA",job title senior data scientist requisition id...,$132367 /yr (est.),3.5
